In [1]:
%cd ..

/home/paulw/projects/TRUSnet-1


In [2]:
from hydra import initialize, compose

initialize(config_path='../configs')

from src.configuration import register_configs

register_configs()

#cfg = compose(overrides=['+datamodule@dm=sl_datamodule', '+split_seed=0'])
cfg = compose(overrides=['+datamodule@dm=concatenated_cores', '+split_seed=2'])

from omegaconf import OmegaConf
OmegaConf.resolve(cfg)

from hydra.utils import instantiate

cfg.dm.splits.merge_test_centers = True
cfg.dm.splits.merge_val_centers = True
cfg.dm.splits.undersample_benign_train = False 
cfg.dm.splits.undersample_benign_eval = False 
cfg.dm.loader_config.num_workers = 0
cfg.dm.loader_config.balance_classes_train = False

from rich import print as pprint
pprint(OmegaConf.to_object(cfg))

/tmp/ipykernel_26622/2693633951.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../configs')
/home/paulw/anaconda3/envs/exact/lib/python3.9/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(


{
    'dm': ExactPatchDMConfig(
        _target_='src.lightning.datamodules.exact_datamodule.PatchesConcatenatedFromCoresData
Module',
        root='/home/mgilani/data/Exact/Exact',
        loader_config=LoaderConfig(
            batch_size=1,
            num_workers=0,
            balance_classes_train=False,
            train_strategy_ddp=False
        ),
        minimum_involvement=0.4,
        splits=SplitsConfig(
            _target_='src.data.exact.splits.Splits',
            cohort_specifier=['CRCEO', 'JH', 'PCC', 'PMCC', 'UVA'],
            train_val_split_seed=2,
            train_val_ratio=0.1,
            test_as_val=False,
            undersample_benign_train=False,
            undersample_benign_eval=False,
            benign_cores_selection_seed=0,
            merge_train_centers=True,
            merge_val_centers=True,
            merge_test_centers=True
        ),
        patch_view_config=PatchViewConfig(
            patch_size=[5, 5],
            patch_strides=[1, 1],
            subpatch_size=[1, 1],
            needle_region_only=True,
            prostate_region_only=False,
            prostate_intersection_threshold=0.9,
            needle_intersection_threshold=0.6
        ),
        patch_transform_train=TransformConfig(
            _target_='src.data.exact.transforms.TransformV3',
            norm=NormConfig(
                _target_='src.data.exact.transforms.Normalize',
                mode='instance',
                type='min-max',
                truncate=True
            ),
            tensor_transform=None,
            us_augmentation=None,
            out_size=[256, 256]
        ),
        patch_transform_eval=TransformConfig(
            _target_='src.data.exact.transforms.TransformV3',
            norm=NormConfig(
                _target_='src.data.exact.transforms.Normalize',
                mode='instance',
                type='min-max',
                truncate=True
            ),
            tensor_transform=None,
            us_augmentation=None,
            out_size=[256, 256]
        )
    ),
    'split_seed': 2
}

In [3]:
dm = instantiate(cfg.dm)

In [ ]:
dm.setup()

Loading Patch Views: 100%|██████████| 1112/1112 [00:41<00:00, 26.86it/s]


In [ ]:
from src.modeling.registry import finetune_linear_all_centers, core_classifier
#model = finetune_linear_all_centers(seed=2, split_seed=2)
model = core_classifier(0, 2)
from src.modeling.seq_models import core_classifier_batch_to_output

In [ ]:
from src.utils.metrics import ClassificationOutputCollector
import torch
from tqdm.notebook import tqdm

@torch.inference_mode()
def test(model, loader): 
    model.eval().cuda()
    collector = ClassificationOutputCollector()

    for batch in tqdm(loader): 
        out = core_classifier_batch_to_output(model, batch, 'cuda')
        collector.collect_batch(out['classification_output'])

    return collector.compute()


out = test(model, dm.test_dataloader())


  0%|          | 0/1112 [00:00<?, ?it/s]

In [ ]:
out['pool_output'].shape

torch.Size([1112, 512])

In [ ]:
#from tqdm.notebook import tqdm
#from ignite.utils import convert_tensor
#from src.utils.metrics import ClassificationOutputCollector
#import torch
#
#@torch.inference_mode()
#def test(model, loader): 
#    model.eval().cuda()
#
#    collector = ClassificationOutputCollector()
#
#    for batch in tqdm(loader):
#        patch, pos, label, metadata = batch 
#        
#        features = model.backbone.get_features(patch.cuda())
#        logits = model.linear_layer(features)
#
#        collector.collect_batch(
#            {
#                'features': features, 
#                'logits': logits, 
#                'position': pos, 
#                'label': label, 
#                **metadata
#            }
#        )
#
#    return collector.compute()
#
#
#
#test(model, dm.test_dataloader())

In [ ]:
import numpy as np 
import os

def all_out_to_numpy(out): 
    for k, v in out.items(): 
        if isinstance(v, torch.Tensor): 
            out[k] = v.numpy()
    return out

def collect_all_and_save_all_latents(name): 
    loaders = {
        'train': dm.train_dataloader(), 
        'val': dm.val_dataloader(), 
        'test': dm.test_dataloader()
    }

    if not os.path.isdir(name):
        os.mkdir(name)
    
    for loader_name, loader in loaders.items(): 
        out = test(model, loader)
        out = all_out_to_numpy(out)
        np.savez(os.path.join(name, f"{loader_name}_latents" ), **out)

collect_all_and_save_all_latents('all_cores_')

  0%|          | 0/7811 [00:00<?, ?it/s]

  0%|          | 0/1013 [00:00<?, ?it/s]

  0%|          | 0/1912 [00:00<?, ?it/s]

: 